In [460]:
from pokebase import *
cache.API_CACHE

'C:\\Users\\erycm\\.cache\\pokebase\\api.cache'

In [461]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
import time

In [462]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "data/pokemondw-e72d35150e00.json"

client = bigquery.Client()

In [464]:
def try_except(fn, excpt):
    try:
        return fn()
    except:
        return excpt

In [465]:
generation = pb.generation(1)

In [467]:
moves = pd.DataFrame(columns=['id', 'name', 'accuracy', 'pp', 'priority', 'power', 'damage_class', 'type', 'use_before1', 'use_before2', 'use_before3', 'use_after1', 'use_after2', 'use_after3'])
for value in generation.moves:
    move = pb.move(value.name)

    row={'id':move.id, 
        'name':move.name, 
        'accuracy':move.accuracy, 
        'pp':move.pp, 
        'priority':move.priority, 
        'power':move.power, 
        'damage_class':move.damage_class.name, 
        'type':move.type.name,
        'use_before1': try_except(lambda: move.contest_combos.normal.use_before[0].name,''), 
        'use_before2': try_except(lambda: move.contest_combos.normal.use_before[1].name,''), 
        'use_before3': try_except(lambda: move.contest_combos.normal.use_before[2].name,''), 
        'use_after1': try_except(lambda: move.contest_combos.normal.use_after[0].name,''), 
        'use_after2': try_except(lambda: move.contest_combos.normal.use_after[1].name,''), 
        'use_after3': try_except(lambda: move.contest_combos.normal.use_after[2].name,'')}
    moves = moves.append(row, ignore_index=True)
    time.sleep(3)
    
moves.head(5)

,id,name,accuracy,pp,priority,power,damage_class,type,use_before1,use_before2,use_before3,use_after1,use_after2,use_after3
0,1,pound,100,35,0,40,physical,normal,double-slap,slam,feint-attack,,,
1,2,karate-chop,100,25,0,50,physical,fighting,,,,focus-energy,,
2,3,double-slap,85,10,0,15,physical,normal,,,,pound,,
3,4,comet-punch,85,15,0,18,physical,normal,,,,,,
4,5,mega-punch,85,20,0,80,physical,normal,,,,focus-energy,mind-reader,


In [468]:

pokemons = pd.DataFrame(columns=['id', 'name', 'base_experience', 'height', 'weight', 'order', 'is_default', 'type1', 'type2', 'hp_effort', 'attack_effort', 'defense_effort', 'special_attack_effort', 'special_defense_effort', 'speed_effort', 'hp_base_stat', 'attack_base_stat', 'defense_base_stat', 'special_attack_base_stat', 'special_defense_base_stat', 'speed_base_stat'])
pokemon_moves = pd.DataFrame(columns=['pokemon_id', 'pokemon_name', 'move_name'])
pokemon_abilities = pd.DataFrame(columns=['pokemon_id', 'pokemon_name', 'is_hidden', 'ability_name'])

for label in generation.pokemon_species:
    pokemon = pb.pokemon(label.name)
    row = {'id': pokemon.id, 
        'name': pokemon.name, 
        'base_experience': pokemon.base_experience, 
        'height': pokemon.height, 
        'weight': pokemon.weight,
        'order': pokemon.order,
        'is_default': pokemon.is_default, 
        'type1': try_except(lambda: pokemon.types[0].type.name, ''), 
        'type2': try_except(lambda: pokemon.types[1].type.name, ''), 
        'hp_effort':try_except(lambda: pokemon.stats[0].effort,''), 
        'attack_effort':try_except(lambda: pokemon.stats[1].effort,''), 
        'defense_effort':try_except(lambda: pokemon.stats[2].effort,''), 
        'special_attack_effort':try_except(lambda: pokemon.stats[3].effort,''), 
        'special_defense_effort':try_except(lambda: pokemon.stats[4].effort,''), 
        'speed_effort':try_except(lambda: pokemon.stats[5].effort,''), 
        'hp_base_stat':try_except(lambda: pokemon.stats[0].base_stat,''), 
        'attack_base_stat':try_except(lambda: pokemon.stats[1].base_stat,''), 
        'defense_base_stat':try_except(lambda: pokemon.stats[2].base_stat,''), 
        'special_attack_base_stat':try_except(lambda: pokemon.stats[3].base_stat,''), 
        'special_defense_base_stat':try_except(lambda: pokemon.stats[4].base_stat,''), 
        'speed_base_stat':try_except(lambda: pokemon.stats[5].base_stat,'') }
    pokemons = pokemons.append(row, ignore_index=True)

    for value in pokemon.moves:
        row = {
            'pokemon_id': pokemon.id,
            'pokemon_name': pokemon.name,
            'move_name': value.move.name}
        pokemon_moves = pokemon_moves.append(row, ignore_index=True)
    
    for value in pokemon.abilities:
        row = {
            'pokemon_id': pokemon.id,
            'pokemon_name': pokemon.name,
            'is_hidden': value.is_hidden,
            'ability_name': value.ability.name}
        pokemon_abilities = pokemon_abilities.append(row, ignore_index=True)
    
    time.sleep(3)

pokemons.head(5)
pokemon_moves.head(5)
pokemon_abilities.head(5)

,pokemon_id,pokemon_name,is_hidden,ability_name
0,1,bulbasaur,False,overgrow
1,1,bulbasaur,True,chlorophyll
2,4,charmander,False,blaze
3,4,charmander,True,solar-power
4,7,squirtle,False,torrent


In [469]:
abilities = pd.DataFrame(columns=['id', 'name', 'is_main_series', 'generation', 'effect', 'short_effect', 'resume_effect'])

for ability_name in pokemon_abilities.ability_name:
    
    effect =''
    short_effect = ''
    flavor_text =''

    ability = pb.ability(ability_name)

    for ef in ability.effect_entries:
        if ef.language.name == 'en':
            effect = ef.effect
            short_effect = ef.short_effect

    for fl in ability.flavor_text_entries:
        if fl.language.name == 'en':
            flavor_text = fl.flavor_text

    row = {
        'id': pokemon.id,
        'name': pokemon.name,
        'is_main_series': value.is_hidden,
        'generation': value.ability.name,
        'effect':effect,
        'short_effect':short_effect,
        'resume_effect':flavor_text}
    
    abilities = abilities.append(row, ignore_index=True)

    time.sleep(3)

abilities.head(5)

,id,name,is_main_series,generation,effect,short_effect,resume_effect
0,76,golem,True,sand-veil,When this Pokémon has 1/3 or less of its HP re...,Strengthens grass moves to inflict 1.5× damage...,Powers up Grass-type moves when the Pokémon’s\...
1,76,golem,True,sand-veil,This Pokémon's Speed is doubled during strong ...,Doubles Speed during strong sunlight.,Boosts the Pokémon’s Speed stat in harsh sunli...
2,76,golem,True,sand-veil,When this Pokémon has 1/3 or less of its HP re...,Strengthens fire moves to inflict 1.5× damage ...,Powers up Fire-type moves when the Pokémon’s H...
3,76,golem,True,sand-veil,"During strong sunlight, this Pokémon has 1.5× ...",Increases Special Attack to 1.5× but costs 1/8...,"Boosts the Sp. Atk stat in harsh sunlight, but..."
4,76,golem,True,sand-veil,When this Pokémon has 1/3 or less of its HP re...,Strengthens water moves to inflict 1.5× damage...,Powers up Water-type moves when the Pokémon’s\...


In [ ]:
all_growth_rates = ['slow', 'medium', 'fast', 'medium-slow', 'slow-then-very-fast', 'fast-then-very-slow']
pokemon_growth_rates = pd.DataFrame(columns=['pokemon_name', 'Growth_id', 'growth_name'])
growth_rates = pd.DataFrame()

for rate in all_growth_rates:
    growth_rate = pb.growth_rate(rate)
    levels = []
    for level in growth_rate.levels:
        levels.append(level.experience)

    for pokemon in growth_rate.pokemon_species:
        row = {
            'pokemon_name': pokemon.name,
            'Growth_id': growth_rate.id,
            'growth_name': growth_rate.name
        }
        pokemon_growth_rates = pokemon_growth_rates.append(row, ignore_index=True)
    growth_rates[rate] = levels

    time.sleep(3)

growth_rates.columns =['slow', 'medium', 'fast', 'medium_slow', 'slow_then_very_fast', 'fast_then_very_slow']

pokemon_growth_rates.head(5)
growth_rates.head(5)


In [ ]:
types = ['normal', 'fighting', 'flying', 'poison', 'ground', 'rock', 'bug','ghost','steel','fire','water','grass','electric','psychic','ice','dragon','dark','fairy','unknown', 'shadow']

json_type = []

for name in types:

    type = pb.type_(name)
    types_damage_list = {}
    
    types_arr = []
    for t in type.damage_relations.no_damage_to:
        # print(t.name)
        types_arr.append({"type":t.name})
    types_damage_list['no_damage_to'] = types_arr

    types_arr = []
    for t in type.damage_relations.half_damage_to:
        # print({"type":t.name})
        types_arr.append({"type":t.name})
    types_damage_list['half_damage_to'] = types_arr

    types_arr = []
    for t in type.damage_relations.double_damage_to:
        # print({"type":t.name})
        types_arr.append({"type":t.name})
    types_damage_list['double_damage_to'] = types_arr

    types_arr = []
    for t in type.damage_relations.no_damage_from:
        # print({"type":t.name})
        types_arr.append({"type":t.name})
    types_damage_list['no_damage_from'] = types_arr

    types_arr = []
    for t in type.damage_relations.half_damage_from:
        # print({"type":t.name})
        types_arr.append({"type":t.name})
    types_damage_list['half_damage_from'] = types_arr

    types_arr = []
    for t in type.damage_relations.double_damage_from:
        # print({"type":t.name})
        types_arr.append({"type":t.name})
    types_damage_list['double_damage_from'] = types_arr

    types_damage_list['type'] = name
    json_type.append(types_damage_list)

In [470]:
job_config = bigquery.LoadJobConfig(
    autodetect=True
)

bigquery_name = 'pokemondw'
table_dataset = 'EL'
table_name = 'moves'
table_id = bigquery_name + "." + table_dataset + "." + table_name
client.load_table_from_dataframe(moves,table_id, job_config=job_config)

bigquery_name = 'pokemondw'
table_dataset = 'EL'
table_name = 'pokemons'
table_id = bigquery_name + "." + table_dataset + "." + table_name
client.load_table_from_dataframe(pokemons,table_id, job_config=job_config)

bigquery_name = 'pokemondw'
table_dataset = 'EL'
table_name = 'pokemon_moves'
table_id = bigquery_name + "." + table_dataset + "." + table_name
client.load_table_from_dataframe(pokemon_moves,table_id, job_config=job_config)

bigquery_name = 'pokemondw'
table_dataset = 'EL'
table_name = 'pokemon_abilities'
table_id = bigquery_name + "." + table_dataset + "." + table_name
client.load_table_from_dataframe(pokemon_abilities,table_id, job_config=job_config)

bigquery_name = 'pokemondw'
table_dataset = 'EL'
table_name = 'abilities'
table_id = bigquery_name + "." + table_dataset + "." + table_name
client.load_table_from_dataframe(abilities,table_id, job_config=job_config)

bigquery_name = 'pokemondw'
table_dataset = 'EL'
table_name = 'pokemon_growth_rates'
table_id = bigquery_name + "." + table_dataset + "." + table_name
client.load_table_from_dataframe(pokemon_growth_rates,table_id, job_config=job_config)

bigquery_name = 'pokemondw'
table_dataset = 'EL'
table_name = 'growth_rates'
table_id = bigquery_name + "." + table_dataset + "." + table_name
client.load_table_from_dataframe(growth_rates,table_id, job_config=job_config)

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("type", "STRING",mode="REQUIRED"),
        bigquery.SchemaField(
            "no_damage_to",
            "RECORD",
            mode="REPEATED",
            fields=[
                bigquery.SchemaField("type", "STRING", mode="NULLABLE"),
            ],
        ),
        bigquery.SchemaField(
            "half_damage_to",
            "RECORD",
            mode="REPEATED",
            fields=[
                bigquery.SchemaField("type", "STRING", mode="NULLABLE"),
            ],
        ),
        bigquery.SchemaField(
            "double_damage_to",
            "RECORD",
            mode="REPEATED",
            fields=[
                bigquery.SchemaField("type", "STRING", mode="NULLABLE"),
            ],
        ),
        bigquery.SchemaField(
            "no_damage_from",
            "RECORD",
            mode="REPEATED",
            fields=[
                bigquery.SchemaField("type", "STRING", mode="NULLABLE"),
            ],
        ),
        bigquery.SchemaField(
            "half_damage_from",
            "RECORD",
            mode="REPEATED",
            fields=[
                bigquery.SchemaField("type", "STRING", mode="NULLABLE"),
            ],
        ),
        bigquery.SchemaField(
            "double_damage_from",
            "RECORD",
            mode="REPEATED",
            fields=[
                bigquery.SchemaField("type", "STRING", mode="NULLABLE"),
            ],
        ),
        
    ],
    # source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
)

bigquery_name = 'pokemondw'
table_dataset = 'EL'
table_name = 'types'
table_id = bigquery_name + "." + table_dataset + "." + table_name

# client.load_table_from_dataframe(growth_rates,table_id, job_config=job_config)
client.load_table_from_json(json_type, table_id, job_config=job_config)

LoadJob<project=pokemondw, location=US, id=9ddc0b8d-cb0e-412e-8cd7-4d0677e25566>